In [1]:
import pandas as pd
import numpy as np

from pyprojroot import here

In [17]:
df = pd.read_csv(here('data/aita_full.csv'))

In [18]:
to_drop_flairs = ['UPDATE',
                  'TL;DR',
                  'Best of 2022',
                  'Open Forum',
                  'META',
                  'Upcoming Talk!!!']

In [19]:
df = df[~df['flair_text'].isin(to_drop_flairs)].reset_index(drop=True).copy()
df = df[df['top_comment'] != '[removed]']
df = df[df['top_comment'] != '[deleted]']
df = df[df['selftext'] != '[removed]']
df = df[df['selftext'] != '[deleted]']
df = df[df['score'] > 25]
df = df[df['top_comment'].str.len() > 15]
df = df.reset_index(drop=True)

In [20]:
df['true_label'] = df['flair_text'].replace({
    'Asshole': 'YTA',
    'Not the A-hole': 'NTA',
    'Everyone Sucks': 'ESH',
    'No A-holes here': 'NAH',
    'Not enough info': 'INFO'})

In [21]:
df['yta_guess'] = df['true_label'].isna() & df['top_comment'].str.contains('YTA| yta | yta.|^yta|Y T A|YWBTA|Yta')
df['nta_guess'] = df['true_label'].isna() & df['top_comment'].str.contains('NTA| nta | nta.|^nta|N T A|YWNBTA|Nta')
df['nah_guess'] = df['true_label'].isna() & df['top_comment'].str.contains('NAH| nah | nah.|^nah')
df['esh_guess'] = df['true_label'].isna() & df['top_comment'].str.contains('ESH| esh | esh.|^esh')
df['info_guess'] = df['true_label'].isna() & df['top_comment'].str.contains('INFO|^Info')

In [22]:
df.loc[df['esh_guess'], 'true_label'] = 'ESH'
df.loc[df['nah_guess'], 'true_label'] = 'NAH'
df.loc[df['info_guess'], 'true_label'] = 'INFO'
df.loc[df['yta_guess'], 'true_label'] = 'YTA'
df.loc[df['nta_guess'], 'true_label'] = 'NTA'

In [23]:
df = df[~df['true_label'].isna()]

In [24]:
df['created_utc'] = pd.to_datetime(df['created_utc'], unit='s')

In [25]:
df = df.drop(['yta_guess',
              'nta_guess',
              'nah_guess',
              'esh_guess',
              'info_guess'], axis=1).sort_values('created_utc', ascending=True).reset_index(drop=True)

In [28]:
df.to_csv(here('data/aita_processed.csv'), index=False)

/opt/homebrew/Caskroom/miniforge/base/envs/nlp/lib/python3.9/site-packages/pyprojroot/pyprojroot.py:51: UserWarning: Path doesn't exist: /Users/psachdeva/projects/moral_foundations_llms/data/aita_processed.csv
  warnings.warn("Path doesn't exist: {}".format(path))
